<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/graph_state/StateValidationWithPydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validation of Graph State using Pydantic framework


## Goals

we're going to illustrate how to use Pydantic and validate state and schema for simple graph scenarios.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph

We will see how [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/) can be used to validate the state schema at run time

Limitations of the Pydantic-based state schema validation:

* currently, the output of the graph will **not** be an instance of a pydantic model

* run-time validation only occurs on inputs into nodes, not on the outputs

* the validation error trace from pydantic does not show which node the error arises in.

We declare the container which holds the public state - `OverallState` and define a single graph node

In [2]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from pydantic import BaseModel

# the overall state of the graph is a public state
class OverallState(BaseModel):
    a: str

def node(state: OverallState):
    return {"a": "goodbye"}

Next we build the graph with state and the defined node

In [4]:
builder = StateGraph(OverallState)
builder.add_node(node) # `node_1` is the first node
builder.add_edge(START, "node")
builder.add_edge("node", END)
graph = builder.compile()

# test the graph with valid input
graph.invoke({"a": "hello"})

{'a': 'goodbye'}

Now let us invoke the same graph with **invalid** input

In [5]:
try:
    graph.invoke({"a": 123}) # should be a string
except Exception as e:
    print("An exception was raised because `a` is an integer rather than a string.")
    print(e)

An exception was raised because `a` is an integer rather than a string.
1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


### Serialization Behavior with Pydantic

We need to understand how serialization works in the following situations:

* passing Pydantic objects as inputs

* receiving outputs from the graph

* working with nested Pydantic models

Below is shown a snippet of code clarifies these behaviors:

First we define two containers which subclass Pydantic's `BaseModel` hence will be subject to seralization

In [6]:
class NestedModel(BaseModel):
    value: str

class ComplexState(BaseModel):
    text: str
    count: int
    nested: NestedModel

We define a node which accepts input of type `ComplexState`, appends something to it and returns the appended and processed string as output

In [7]:
def process_node(state: ComplexState):
    # Node receives a validated Pydantic object
    print(f"Input state type: {type(state)}")
    print(f"Nested type: {type(state.nested)}")
    # return a dicntionary update
    return {"text": state.text + " processed", "count": state.count + 1}

Next we build and compile the graph model

In [8]:
builder = StateGraph(ComplexState)
builder.add_node("process", process_node)
builder.add_edge(START, "process")
builder.add_edge("process", END)
graph = builder.compile()

Create a Pydantic instance for input and invoke the graph with the new Pydantic instance

In [10]:
input_state = ComplexState(text="hello", count=0, nested=NestedModel(value="test"))
print(f"Input object type: {type(input_state)}")

result = graph.invoke(input_state)
print(f"Output type: {type(result)}")
print(f"Output content: {result}")

Input object type: <class '__main__.ComplexState'>
Input state type: <class '__main__.ComplexState'>
Nested type: <class '__main__.NestedModel'>
Output type: <class 'dict'>
Output content: {'text': 'hello processed', 'count': 1, 'nested': NestedModel(value='test')}


Finally, convert back the result to Pydantic model

In [11]:
output_model = ComplexState(**result)
print(f'Converted back to Pydantic: {type(output_model)}')

Converted back to Pydantic: <class '__main__.ComplexState'>


### Runtime Type Coercion with Pydantic

We will clarify on the runtime coercion behavior of Pydantic BaseModel in relevant for us scenarios via the code snippet below

In [ ]:
class CoercionExample(BaseModel):
    # Pydantic coerces string numbers to integers
    number: int
